# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function based on some assumptions about our business. 

In [1]:
import evalml
from evalml.objectives import FraudDetection
from evalml.preprocessing import split_data

## Load Transactions

In [13]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


## Define Objective

In [14]:
fraud_objective = FraudDetection(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [15]:
# select numeric data before running AutoClassifer
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=20)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Detection. Lower score is better.

Searching up to 20 pipelines. Will stop searching for new pipelines after 120 seconds.

Possible model types: xgboost, linear_model, random_forest

Testing Random Forest w/ imputation:  70%|███████   | 14/20 [02:07<01:41, 17.00s/it]              

Max time elapsed. Stopping search early.
Testing Random Forest w/ imputation:  70%|███████   | 14/20 [02:07<00:54,  9.07s/it]

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,5,LogisticRegressionPipeline,2575489.89,False,"{'penalty': 'l2', 'C': 8.123565600467177, 'imp..."
1,1,LogisticRegressionPipeline,2575923.89,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
2,3,LogisticRegressionPipeline,2575961.12,False,"{'penalty': 'l2', 'C': 0.5765626434012575, 'im..."
3,2,LogisticRegressionPipeline,7123230.87,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
4,8,LogisticRegressionPipeline,8333754.92,False,"{'penalty': 'l2', 'C': 8.362426847738403, 'imp..."
5,13,RFPipeline,9932174.80,False,"{'n_estimators': 715, 'max_depth': 487, 'imput..."
6,6,RFPipeline,10203436.96,False,"{'n_estimators': 569, 'max_depth': 630, 'imput..."
7,11,RFPipeline,11112779.91,False,"{'n_estimators': 609, 'max_depth': 71, 'impute..."
8,0,XGBoostPipeline,11610654.75,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
9,4,XGBoostPipeline,11610654.75,False,"{'eta': 0.38438170729269994, 'min_child_weight..."


to select the best pipeline we can run

In [7]:
pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [8]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [9]:
clf.describe_pipeline(0)

************************
* Pipeline Description *
************************

Pipeline Name: XGBoost w/ imputation
Model type: xgboost
Objective: Fraud Detection (lower is better)
Total training time (including CV): 4.1 seconds

Parameters
• eta: 0.5928446182250184
• min_child_weight: 8.598391737229157
• max_depth: 4
• impute_strategy: most_frequent
• percent_features: 0.6273280598181127

Cross Validation
        F1  Precision  Recall   AUC  Log Loss  Fraud Detection
0    0.813      1.000   0.813 0.842     1.653     10811263.500
1    0.812      1.000   0.812 0.842     1.658     10677483.750
2    0.799      1.000   0.799 0.833     1.756     11610654.750
mean 0.808      1.000   0.808 0.839     1.689     11033134.000
std  0.008      0.000   0.008 0.006     0.058       504600.751


## Test on hold out

In [10]:
pipeline.fit(X_train, y_train)

In [12]:
pipeline.score(X_holdout, y_holdout)

138019171.48